In [ ]:
# Install required packages
#!pip install pandas
#!pip install indian-names
#!pip install random-word
#!pip install psycopg2
#!pip install faker
#!pip install pyarrow
#!pip install sqlalchemy
#!pip install scikit-learn

In [ ]:
# Imports
import pandas as pd
import indian_names
import random
import psycopg2
from random_word import RandomWords
from datetime import datetime, timedelta
import pyarrow as pa
import pyarrow.parquet as pq
import os
import json
from collections import Counter
from faker import Faker
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize Faker for generating fake data
fake = Faker()


In [ ]:
import pandas as pd

def generate_weather_data(city, start_date, iterations):
    """
    Generate weather data for a given city over a specified duration.

    Args:
        city (str): The name of the city.
        start_date (datetime): The start date for generating weather data.
        iterations (int): The number of iterations (time intervals) to generate weather data.

    Returns:
        list: A list of dictionaries containing weather data for the given city.
    """
    # Temperature ranges and weather conditions for different seasons (in Celsius)
    summer_temp_range = (25, 40)
    summer_conditions = ['sunny', 'partly cloudy']
    summer_wind_speed_range = (5, 15)  # in km/h
    summer_humidity_range = (30, 60)   # in percentage
    
    rainy_temp_range = (20, 30)
    rainy_conditions = ['rainy', 'partly cloudy', 'heavy rain', 'thunderstorm', 'zero visibility']
    rainy_wind_speed_range = (10, 25)  # in km/h
    rainy_humidity_range = (60, 90)    # in percentage
    
    winter_temp_range = (10, 25)
    winter_conditions = ['clear', 'partly cloudy', 'foggy']
    winter_wind_speed_range = (5, 10)   # in km/h
    winter_humidity_range = (20, 50)    # in percentage
    
    # Initialize list to store weather data
    weather_data = []
    
    # Loop for every time interval
    for _ in range(iterations):
        # Randomly select season based on month
        month = start_date.month
        if 3 <= month <= 5:
            season = 'summer'
        elif 6 <= month <= 9:
            season = 'rainy'
        else:
            season = 'winter'
        
        # Randomly select weather condition, temperature, wind speed, and humidity based on season
        if season == 'summer':
            temperature = round(random.uniform(*summer_temp_range), 2)
            weather_condition = random.choice(summer_conditions)
            wind_speed = round(random.uniform(*summer_wind_speed_range), 2)
            humidity = round(random.uniform(*summer_humidity_range), 2)
        elif season == 'rainy':
            temperature = round(random.uniform(*rainy_temp_range), 2)
            weather_condition = random.choices(rainy_conditions, weights=[0.3, 0.2, 0.2, 0.2, 0.1])[0]
            wind_speed = round(random.uniform(*rainy_wind_speed_range), 2)
            humidity = round(random.uniform(*rainy_humidity_range), 2)
        else:
            temperature = round(random.uniform(*winter_temp_range), 2)
            weather_condition = random.choices(winter_conditions, weights=[0.4, 0.4, 0.2])[0]
            wind_speed = round(random.uniform(*winter_wind_speed_range), 2)
            humidity = round(random.uniform(*winter_humidity_range), 2)
        
        # Append weather data to the list
        weather_data.append({
            'city': city,
            'time': start_date,
            'condition': weather_condition,
            'temperature': temperature,
            'wind_speed': wind_speed,
            'humidity': humidity
        })
        
        # Increment time by 10 minutes for the next iteration
        start_date += timedelta(minutes=10)
    
    return weather_data


def training_dataset(start_date, iterations):
    """
    Generate a training dataset of weather data for multiple cities over a specified duration.

    Args:
        start_date (datetime): The start date for generating weather data.
        iterations (int): The number of time intervals to generate weather data.

    Returns:
        pandas.DataFrame: A DataFrame containing weather data for multiple cities.
    """
    # Initialize an empty DataFrame to store weather data
    weather_df = pd.DataFrame()

    # Cities for which weather data will be generated
    cities = ['Mumbai', 'Delhi', 'Pune', 'Hyderabad', 'Bangalore']

    # Generate weather data for each city and append to DataFrame
    for city in cities:
        city_weather_data = generate_weather_data(city, start_date, iterations)
        weather_df = weather_df._append(city_weather_data, ignore_index=True)

    return weather_df


In [ ]:
def determine_weather_category(row):
    """
    Determine the weather category based on temperature, humidity, and wind speed thresholds.

    Args:
        row (pandas.Series): A row of data containing weather information.

    Returns:
        str: The weather category ('good' or 'bad').
    """
    # Define thresholds for weather conditions
    temperature_threshold = 25  # Example threshold for temperature
    humidity_threshold = 70     # Example threshold for humidity
    wind_speed_threshold = 15    # Example threshold for wind speed
    
    # Check temperature, humidity, and wind speed to determine weather category
    if row['temperature'] > temperature_threshold and row['humidity'] < humidity_threshold and row['wind_speed'] < wind_speed_threshold:
        return 'good'
    elif row['temperature'] < temperature_threshold and row['humidity'] > humidity_threshold and row['wind_speed'] > wind_speed_threshold:
        return 'bad'
    elif row['condition'] in ['thunderstorm', 'foggy', 'zero visibility']:
        return 'bad'
    else:
        return 'good'


def data_model(data):
    """
    Train a Random Forest classifier based on weather data.

    Args:
        data (list of tuples): The weather data.

    Returns:
        RandomForestClassifier: The trained classifier model.
    """
    # Create DataFrame from the provided data
    df = pd.DataFrame(data, columns=['city', 'time', 'condition', 'temperature', 'wind_speed', 'humidity'])

    # Apply the 'determine_weather_category' function to create a new weather category column
    df['weather_category'] = df.apply(determine_weather_category, axis=1)

    # Print unique cities in the dataset
    print(df['city'].unique())

    # Select features (temperature, wind speed, humidity) and target variable (weather category)
    X = df[['temperature', 'wind_speed', 'humidity']]
    y = df['weather_category']

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Train Random Forest classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = clf.predict(X_test)

    # Evaluate model accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    return clf


In [ ]:
def write_to_postgress(df, table_name):
    """
    Write DataFrame data to a PostgreSQL table.

    Args:
    df (pandas.DataFrame): The DataFrame containing the data to be written.
    table_name (str): The name of the PostgreSQL table to write the data to.
    """
    # PostgreSQL connection parameters
    username = 'sv_pgdev'
    password = 'Zathura@5499'
    host = 'dev-zathura-edw.postgres.database.azure.com'
    port = '5432'
    database = 'Landing'
    
    try:
        # Establish a connection to the PostgreSQL server
        conn = psycopg2.connect(
            dbname=database,
            user=username,
            password=password,
            host=host,
            port=port
        )
    
        # Create a cursor object
        cursor = conn.cursor()
    
        # Convert DataFrame to list of tuples
        rows = [tuple(x) for x in df.to_numpy()]

        # Construct the placeholder string
        placeholders = ', '.join(['%s'] * len(df.columns))
    
        # Construct the INSERT INTO statement with parameterized query
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
    
        # Execute the INSERT statement with executemany
        cursor.executemany(insert_query, rows)

        # Commit the transaction
        conn.commit()
    
    except psycopg2.Error as e:
        print("Error: Could not connect to PostgreSQL server:", e)
    
    finally:
        # Close the cursor and connection
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()


In [ ]:
def generate_aadhar_number():
    """
    Generate a random Aadhar number.

    Returns:
    str: A randomly generated Aadhar number.
    """
    # Generate three groups of four-digit numbers and join them with spaces
    aadhar_number = ' '.join(''.join(map(str, random.sample(range(10), 4))) for x in range(3))
    
    return aadhar_number


In [ ]:
def generate_passport_number():
    """
    Generate a random passport number.

    Returns:
    str: A randomly generated passport number.
    """
    alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    # Choose a random letter from the alphabet
    letter = random.choices(alphabets)[0]

    # Generate the rest of the passport number
    number = (letter +
              str(random.randint(1, 9)) +
              str(random.randint(0, 9)) +
              ''.join(map(str, random.sample(range(10), 4))) +
              str(random.randint(1, 9)))

    return number


In [ ]:
def generate_phone_number():
    """
    Generate a random phone number.

    Returns:
    str: A randomly generated phone number.
    """
    phone_number = ''

    # The first digit of the phone number should be between 7 to 9
    phone_number += str(random.randint(7, 9))

    # Generate the rest of the digits for the phone number
    for i in range(1, 10):
        phone_number += str(random.randint(0, 9))

    return phone_number


In [ ]:
def generate_email(choices):
    """
    Generate a random email address.

    Parameters:
    choices (list): List of choices for generating the email address.

    Returns:
    str: A randomly generated email address.
    """
    # Define number choices for email address
    number_choice = [''.join(map(str, random.sample(range(10), x))) for x in range(2, 5)]

    # Define domain choices for email address
    domain_choice = ['gmail.com', 'yahoo.com', 'hotmail.com']

    # Generate and return a random email address
    return (
        random.choices(choices, weights=[0.5, 0.3, 0.1, 0.1])[0] +
        '.' +
        random.choices(choices, weights=[0.2, 0.6, 0.1, 0.1])[0] +
        random.choice(number_choice) +
        '@' +
        random.choice(domain_choice)
    )


In [ ]:
def random_with_N_digits(n):
    """
    Generate a random integer with N digits.

    Parameters:
    n (int): The number of digits for the random integer.

    Returns:
    int: A random integer with N digits.
    """
    # Define the range for the random integer
    range_start = 10**(n-1)
    range_end = (10**n) - 1

    # Generate and return a random integer within the defined range
    return random.randint(range_start, range_end)


In [ ]:
def get_boardingweights(start_date):
    """
    Generate boarding weights based on the month.

    Parameters:
    month (int): The month for which boarding weights are to be generated.

    Returns:
    list: A list containing boarding weights for boarding and not boarding.
    """
    diff = 0
    month = start_date.month
    # Adjust weights for boarding during peak months (June to September)
    if 6 <= month <= 9:
        diff = random.uniform(0.1, 0.3)

    # Generate a random number within a range
    random_num = random.uniform(0.1, 0.4 + diff)

    # Define choices and weights for random selection
    choices = [0, random_num]
    weights = [0.4, 0.6]

    # Randomly choose a value based on weights
    result = random.choices(choices, weights=weights)[0]

    # Return the list of boarding weights
    return [1 - result, result]


In [ ]:
class Passenger:
    def __init__(self):
        gender = random.choice(["male", "female"])  # Randomly select gender
        self.first_name = indian_names.get_first_name(gender)  # Get first name based on gender
        self.last_name = indian_names.get_last_name()  # Get last name
        self.age = random.randint(5, 80)  # Generate random age between 5 and 80

        # Assign gender considering 'Others' option for passengers below 18
        if self.age > 18:
            self.gender = random.choices([gender, "Others"], weights=[0.95, 0.05])[0]
        else:
            self.gender = gender

        # For passengers below 15 years, set contact number, email ID, and last name to empty string
        if self.age < 15:
            self.contact_number = ""
            self.email_id = ""
            self.last_name = ""
        else:
            # Generate contact number and email ID
            self.contact_number = generate_phone_number()
            self.email_id = generate_email([self.first_name, self.last_name, RandomWords().get_random_word(), RandomWords().get_random_word()])

        # Generate Aadhar number and choose between generating a passport number or leaving it blank
        self.aadhar_number = generate_aadhar_number()
        self.passport_number = random.choice([generate_passport_number(), ""])


In [ ]:
class Airplanes:
    def __init__(self, airplane_type, airplane_name, frequency):
        if airplane_type == 'Boeing 777X':
            # Attributes for Boeing 777X
            self.fuel_capacity = 350410  # Fuel capacity in lbs
            self.seats = 54  # Total number of seats
            self.classes = {
                "Economy": {"seat_type": "D", "seats": 20, "window_seats": 9},
                "Premium_Economy": {"seat_type": "C", "seats": 14, "window_seats": 8},
                "Business": {"seat_type": "B", "seats": 12, "window_seats": 6},
                "First_class": {"seat_type": "A", "seats": 8, "window_seats": 3}
            }  # Classes available in the airplane
            self.range = 16190  # Range in km
            self.window_seats = 26  # Total number of window seats
            self.base_price = 5000  # Base price for the airplane
        elif airplane_type == 'Airbus A220':
            # Attributes for Airbus A220
            self.fuel_capacity = 21805  # Fuel capacity in lbs
            self.seats = 40  # Total number of seats
            self.classes = {
                "Economy": {"seat_type": "D", "seats": 25, "window_seats": 12},
                "Premium_Economy": {"seat_type": "C", "seats": 15, "window_seats": 7}
            }  # Classes available in the airplane
            self.range = 6390  # Range in km
            self.window_seats = 19  # Total number of window seats
            self.base_price = 3000  # Base price for the airplane
        elif airplane_type == 'Airbus A320':
            # Attributes for Airbus A320
            self.fuel_capacity = 27200  # Fuel capacity in lbs
            self.seats = 45  # Total number of seats
            self.classes = {
                "Economy": {"seat_type": "D", "seats": 15, "window_seats": 6},
                "Premium_Economy": {"seat_type": "C", "seats": 12, "window_seats": 5},
                "Business": {"seat_type": "B", "seats": 10, "window_seats": 4},
                "First_class": {"seat_type": "A", "seats": 8, "window_seats": 2}
            }  # Classes available in the airplane
            self.range = 6150  # Range in km
            self.window_seats = 17  # Total number of window seats
            self.base_price = 3500  # Base price for the airplane

        self.name = airplane_name  # Name of the airplane
        self.airplane_id = airplane_name[:3] + '-' + airplane_type[:3]  # Unique ID for the airplane
        self.type = airplane_type  # Type of the airplane
        self.current_fuel = 0  # Current fuel level of the airplane
        self.frequency = frequency  # Frequency of flights for the airplane


In [ ]:
class Airport:
    def __init__(self, airport_name, city_name):
        self.name = airport_name  # Name of the airport
        self.code = city_name[:3].upper()  # Code for the airport, first three letters of city name
        self.city = city_name  # Name of the city
        self.airplanes = set()  # Set to store airplanes associated with the airport


In [ ]:
class Flight:
    def __init__(self, airplane, source_airport, destination_airport, source_city, destination_city, flight_id, start_time, hour, agent,model):
        # Initialize flight attributes
        self.id = flight_id
        self.source = source_airport
        self.destination = destination_airport
        self.source_city = source_city
        self.destination_city = destination_city
        self.passengers = []
        self.payments = []
        self.agents = agent
        self.status = None
        self.comments = None
        
        # Extracting data from airplane object
        curr_airplane = airplane
        self.total_seats = curr_airplane.seats
        self.flight_code = curr_airplane.airplane_id
        self.classes = curr_airplane.classes
        self.base_price = curr_airplane.base_price
        self.estimated_departure_time = start_time
        self.estimated_arrival_time = start_time + timedelta(hours=hour)
        
        # Generate random departure and arrival times
        self.departure_time = random.choice([(start_time + timedelta(minutes=-random.randint(0, 3))),
                                             (start_time + timedelta(minutes=random.randint(0, 15)))])
        self.arrival_time = random.choice([(self.departure_time + timedelta(hours=hour) + timedelta(minutes=-random.randint(0, 15))),
                                           (self.departure_time + timedelta(hours=hour) + timedelta(minutes=random.randint(0, 15)))])
        
        # Generate weather data for destination city
        input_data = generate_weather_data(self.destination_city, self.arrival_time + timedelta(minutes=-30), 6)
        df = pd.DataFrame(input_data, columns=['city', 'time', 'condition', 'temperature', 'wind_speed', 'humidity'])
        df = df[['temperature', 'wind_speed', 'humidity']]
        
        # Use model to predict flight status based on weather data
        output_data = model.predict(df)
        counts = Counter(output_data)
        most_common_value = counts.most_common(1)[0][0]
        
        # Set flight status based on prediction
        if most_common_value == 'bad':
            self.status = 'Cancelled'
            self.departure_time = None
            self.arrival_time = None
        else:
            self.status = 'Landed'
       
    # Method to get flight data
    def get_flight_data(self):
        return {
            "flight_id": self.id,
            "flight_code": self.flight_code,
            "source_airport": self.source,
            "destination_airport": self.destination,
            "source_city": self.source_city,
            "destination_city": self.destination_city,
            "estimated_departure_time": self.estimated_departure_time,
            "departure_time": self.departure_time,
            "estimated_arrival_time": self.estimated_arrival_time,
            "arrival_time": self.arrival_time,
            "passenger_data": self.passengers,
            "total_passengers": len(self.passengers),
            "status": self.status
        }
        
    # Method to fill the flight with passengers
    def fill_flight(self):
        base_price = self.base_price  # Get base price for the flight
        flight_classes = self.classes  # Get classes present in the flight
        last_passenger = None
        last_payment = None

        
        # Populate flight seat by seat
        for flight_class in list(flight_classes.keys()):
            total_seats = flight_classes[flight_class]["seats"]
            window_seats = flight_classes[flight_class]["window_seats"]
            seat_type = flight_classes[flight_class]["seat_type"]


            boarding_weights = get_boardingweights(self.estimated_departure_time)
            while total_seats > 0:
                
                
                boarding = random.choices(["Yes", "No"], weights=boarding_weights)[0]  # Decide whether seat will be filled or not
                
                if boarding == "Yes":
                    # Calculate fare based on seat type
                    cash = base_price
                    seating_class = 'Economy'
                    if seat_type == 'A':
                        cash += base_price * 0.4
                        seating_class = 'First-class'
                    elif seat_type == 'B':
                        cash += base_price * 0.3
                        seating_class = 'Business'
                    elif seat_type == 'C':
                        cash += base_price * 0.2
                        seating_class = 'Premium-Economy'
    
                    seat_number = seat_type + '-' + str(total_seats)
                    
                    # Determine if it's a window seat and adjust fare accordingly
                    window = random.choices(["Yes", "No"], weights=[0.4, 0.6])[0]
                    if window_seats > 0 and window:
                        seat_number += '(W)'
                        cash += base_price * 0.1
                        window_seats -= 1

                    # Add additional charges
                    cash += random.choices([0, boarding_weights[1] * 100], weights=[0.6, 0.4])[0]
                    
                    # Generate passenger and payment data
                    passenger = Passenger()

                    #Handline base case since last_payment and last_passenger are None type - if we receive first passenger with age < 18
                    if(len(self.passengers) == 0 and passenger.age < 18): 
                        passenger.age = 21 
                    
                    payment_mode = random.choices(['Credit/Debit Card', 'UPI', 'Cash'], weights=[0.4, 0.59, 0.01])[0]
                    transaction_id = -1
                    if payment_mode != 'Cash':
                        transaction_id = random_with_N_digits(10)

                    time_of_payment = self.estimated_departure_time + timedelta(hours=-random.randint(4, 150)) + timedelta(minutes=-random.randint(0, 55)) 
                    
                    agent = random.choice(self.agents)
                    if transaction_id == -1:
                        agent = "self"
                    
                    # Handling passengers under 18
                    if passenger.age < 18:      
                        passenger.last_name = last_passenger["last_name"]
                        passenger.contact_number = last_passenger["contact_number"]
                        passenger.email_id = last_passenger["email_id"]
                        transaction_id = last_passenger["bookingID"]

                    passenger_data = {
                        "first_name": passenger.first_name,
                        "last_name": passenger.last_name,
                        "age": passenger.age,
                        "gender": passenger.gender,
                        "contact_number": passenger.contact_number,
                        "email_id": passenger.email_id,
                        "aadhar_number": passenger.aadhar_number,
                        "passport_number": passenger.passport_number,
                        "seating_class": seating_class,
                        "seat": seat_number,
                        "bookingID": transaction_id
                    }
                    
                    # Handling passengers under 18
                    payer = Passenger()
                    while payer.age <= 18:
                        payer = Passenger()

                    if passenger.age < 18:
                        payer.first_name = last_payment["first_name"]
                        payer.last_name = last_payment["last_name"]
                        payer.contact_number = last_payment["contact_number"]
                        payer.email_id = last_payment["email_id"]
                        time_of_payment = last_payment["time_of_payment"]
                        payment_mode = last_payment["mode_of_payment"]
                        if last_passenger["seat"][0] == seat_number[0]:
                            cash = last_payment["Amount"]
                        agent = last_payment["Agent"]
                    else:
                        payer_first_name = random.choices([passenger.first_name, payer.first_name], weights=[0.7, 0.3])[0]
                        if passenger.first_name != payer_first_name:
                            payer.last_name = random.choices([passenger.last_name, payer.last_name], weights=[0.7, 0.3])[0]     
                        else:
                            payer.first_name = passenger.first_name
                            payer.last_name = passenger.last_name
                            payer.contact_number = passenger.contact_number
                            payer.email_id = passenger.email_id
                                
                    payment_transaction = {
                        "flight_id": self.id,
                        "first_name": payer.first_name,
                        "last_name": payer.last_name,
                        "contact_number": payer.contact_number,
                        "email_id": payer.email_id,
                        "seat": seat_number,
                        "mode_of_payment": payment_mode,
                        "time_of_payment": time_of_payment,
                        "Amount": cash,
                        "TransactionID": transaction_id,
                        "Agent": agent
                    }

                    # Append passenger and payment data
                    self.passengers.append(passenger_data) 
                    self.payments.append(payment_transaction) 

                    if passenger_data["age"] >= 18:
                        last_passenger = passenger_data
                        last_payment = payment_transaction

                total_seats -= 1

        # Write payment data to database
        df = pd.DataFrame(self.payments)
        df['time_of_payment'] = df['time_of_payment'].astype(str)
        write_to_postgress(df, 'payments')


In [ ]:
class Zathura:
    # Constructor
    def __init__(self,id,start_date_str, end_date_str,model):
        # Hardcoded values and data structures initialization
        self.global_flight_id = id
        self.aircraft_models = {
            'Boeing 777X': ['Garuda', 'Vimana', 'Vayu', 'Ananta'],
            'Airbus A220': ['Agni', 'Astra', 'Vajra'],
            'Airbus A320': ['Rudra', 'Marut']
        }
        self.cities = ["Mumbai", "Delhi", "Pune", "Banglore", "Hydrebad"]
        self.airports = ["Chhatrapati Shivaji Maharaj International Airport", "Indira Gandhi International Airport",
                         "Pune Airport", "HAL Bangalore International Airport", "Rajiv Gandhi International Airport"]

        self.airports_city_link = {"Mumbai": ["Chhatrapati Shivaji Maharaj International Airport"],
                                   "Delhi": ["Indira Gandhi International Airport"],
                                   "Pune": ["Pune Airport"], "Banglore": ["HAL Bangalore International Airport"],
                                   "Hydrebad": ["Rajiv Gandhi International Airport"]}
        self.airport_data = {}
        self.flights_data = []
        self.airplane_data = []
        self.airplane_initials = {
            'Garuda': {'frequency': '1357', 'source': 'Pune Airport', 'destination': 'Indira Gandhi International Airport', 'timings': [7, 12, 17, 23], 'index': 0, 'hours': 2},
            'Vimana': {'frequency': '1357', 'source': 'Chhatrapati Shivaji Maharaj International Airport', 'destination': 'Indira Gandhi International Airport', 'timings': [6, 12, 18], 'index': 0, 'hours': 2},
            'Vayu': {'frequency': '246', 'source': 'Rajiv Gandhi International Airport', 'destination': 'Indira Gandhi International Airport', 'timings': [9, 13, 19, 23], 'index': 0, 'hours': 3},
            'Ananta': {'frequency': '246', 'source': 'Indira Gandhi International Airport', 'destination': 'HAL Bangalore International Airport', 'timings': [6, 11, 17], 'index': 0, 'hours': 4},
            'Agni': {'frequency': '2345', 'source': 'HAL Bangalore International Airport', 'destination': 'Pune Airport', 'timings': [5, 10, 15, 20], 'index': 0, 'hours': 3},
            'Astra': {'frequency': '2345', 'source': 'HAL Bangalore International Airport', 'destination': 'Chhatrapati Shivaji Maharaj International Airport', 'timings': [7, 12, 17, 23], 'index': 0, 'hours': 3},
            'Vajra': {'frequency': '234567', 'source': 'Pune Airport', 'destination': 'Rajiv Gandhi International Airport', 'timings': [6, 11, 16, 21], 'index': 0, 'hours': 2},
            'Rudra': {'frequency': '1234567', 'source': 'Chhatrapati Shivaji Maharaj International Airport', 'destination': 'Rajiv Gandhi International Airport', 'timings': [12, 17], 'index': 0, 'hours': 2},
            'Marut': {'frequency': '1234567', 'source': 'Chhatrapati Shivaji Maharaj International Airport', 'destination': 'Pune Airport', 'timings': [6, 12, 18], 'index': 0, 'hours': 1}
        }
        self.agents = ['Zathura-website', 'Zathura-mobilebooking', 'VoyageNest.com', 'BookifyNow.com',
                       'OneStopAirtickets.com', 'TourEase.com']
        self.start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        self.end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
        self.weather_model = model

    # Setting airport data
    def set_airport_data(self):
        for city in self.cities:
            for airport in self.airports_city_link[city]:
                self.airport_data[airport] = Airport(airport, city)

    # Setting airplane data
    def set_airplane_data(self):
        for model in self.aircraft_models:
            for model_name in self.aircraft_models[model]:
                self.airplane_data.append(Airplanes(model, model_name, self.airplane_initials[model_name]['frequency']))

    # Setting airplanes to an initial airport
    def set_inital_airplanes_to_airports(self):
        for airplane in self.airplane_data:
            airport = self.airplane_initials[airplane.name]['source']
            self.airport_data[airport].airplanes.add(airplane)

    # Load initial tables
    def load_initaltables(self):
        # Flights
        flights_df = pd.DataFrame(self.airplane_initials)
        flights_df = flights_df.transpose()
        flights_df.drop(columns=['index', 'hours'], inplace=True)
        flights_df.reset_index(inplace=True)
        flights_df.rename(columns={'index': 'name'}, inplace=True)
        write_to_postgress(flights_df, 'flight_schedules')

        # Airplanes
        data = []

        for airplane in self.airplane_data:
            data.append({
                "airplaneId": airplane.airplane_id,
                "name": airplane.name,
                "type": airplane.type,
                "total_seats": airplane.seats,
                "classes": airplane.classes,
                "fuel_capacity": airplane.fuel_capacity,
                "range": airplane.range
            })

        airplane_df = pd.DataFrame(data)
        airplane_df['classes'] = airplane_df['classes'].apply(json.dumps)
        write_to_postgress(airplane_df, 'airplanes')

    # Schedule flights
    def schedule_flights(self):
        while self.start_date <= self.end_date:
            curr_weekday = self.start_date.weekday() + 1
            destination_tracker = {airport_name: set() for airport_name in self.airports}
            source_tracker = {airport_name: set() for airport_name in self.airports}
            flag = True
            
            # Go through all airports and schedule flight for each airplane present in the airport
            for airport in self.airports:
                source_airport = airport
                source_airport_data = self.airport_data[source_airport]
                source_airplanes = source_airport_data.airplanes

                for airplane in source_airplanes:
                    freq = self.airplane_initials[airplane.name]['frequency']
                    size = len(self.airplane_initials[airplane.name]['timings'])
                    index = self.airplane_initials[airplane.name]['index']
                    
                    if str(curr_weekday) in freq and index < size:
                        flag = False
                        hour = self.airplane_initials[airplane.name]['hours']
                        time = self.airplane_initials[airplane.name]['timings'][index]
                        destination_airport = self.airplane_initials[airplane.name]['destination']

                        source_city = source_airport_data.city
                        destination_city = self.airport_data[destination_airport].city

                        print(f"{airplane.name} => {self.airplane_initials[airplane.name]['source']} => {self.airplane_initials[airplane.name]['destination']}")
                        #print(f"{airplane.name} => {self.airplane_initials[airplane.name]} => {self.airplane_initials[airplane.name]['destination']} - {(self.start_date + timedelta(hours=time))}")

                        flight = Flight(airplane, source_airport, destination_airport, source_city, destination_city, self.global_flight_id,
                                        (self.start_date + timedelta(hours=time)), hour, self.agents,self.weather_model)
                        flight.fill_flight()
                        self.flights_data.append(flight.get_flight_data())

                        self.airplane_initials[airplane.name]['index'] += 1

                        source_tracker[source_airport].add(airplane)
                        destination_tracker[destination_airport].add(airplane)

                        self.airplane_initials[airplane.name]['destination'] = self.airplane_initials[airplane.name]['source']
                        self.airplane_initials[airplane.name]['source'] = destination_airport

                        self.global_flight_id += 1

                        df = pd.DataFrame(self.flights_data)
                        df['departure_time'] = df['departure_time'].astype(str)
                        df['arrival_time'] = df['arrival_time'].astype(str)
                        df['passenger_data'] = df['passenger_data'].apply(json.dumps)

                        write_to_postgress(df, 'flights')
                        self.flights_data = []

            # Fixing trackers for the next run
            for airport in self.airports:
                self.airport_data[airport].airplanes = self.airport_data[airport].airplanes | destination_tracker[airport]
                self.airport_data[airport].airplanes = self.airport_data[airport].airplanes - source_tracker[airport]

            if flag:
                self.start_date = self.start_date + timedelta(days=1)
                for airplane in self.airplane_data:
                    self.airplane_initials[airplane.name]['index'] = 0

    def get_print(self):
        for airport in self.airports:
            curr_airport = self.airport_data[airport]
            print(f"{curr_airport.name}- ")
            for ap in curr_airport.airplanes:
                print(f" \t\t {ap.name}")

    def show_data(self):
        flight_schedules_df = pd.DataFrame(self.flights_data)
        display(flight_schedules_df)



In [ ]:
    # Define the start date for the data collection
    start_date = datetime.strptime('2022-01-01', "%Y-%m-%d")

    # Calculate the number of iterations based on the duration of data collection
    # In this case, it's calculated for 2 years (365 days/year), 6 times a day
    iterations = 24 * 365 * 2 * 6

    # Generate training dataset starting from the specified start date and for the specified number of iterations
    data = training_dataset(start_date, iterations)

    # Train the model using the generated training dataset
    model = data_model(data)


In [ ]:
def main(model):

    # Create an instance of the Zathura class
    zathura = Zathura(100001,'2022-06-01','2023-01-01',model)

    # Set airplane data
    zathura.set_airplane_data()

    # Set airport data
    zathura.set_airport_data()

    # Set initial airplanes to airports
    zathura.set_inital_airplanes_to_airports()

    # Load initial tables
    zathura.load_initaltables()

    # Print airport and airplane information
    zathura.get_print()

    # Schedule flights
    zathura.schedule_flights()

In [ ]:
if __name__ == "__main__":
    main(model)